In [45]:
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [47]:
#Define the predictor datasets
predictors = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

#Decide which dataset you want to predict
response_number = 1

In [48]:
#Get your response dataset
response = [col for col in DF if col.startswith(predictors[response_number])]
response.append("patientID")
response_df = DF[response]

In [49]:
# Choose your image size and number of features we are predicting
pixels = 40
features = len(response)-1

In [50]:
predictors.append(tuple(predictors))

In [51]:
#npredictors = len(predictors)
#book = []
#for perplexity in range(5,150,5):
#    albums, patient_IDs = zip(*Parallel(n_jobs=npredictors)(delayed(butterfly.album.create_album)
#                                                    (DF, predictors[al], pixels, perplexity) 
#                                                    for al in tqdm(range(npredictors))))
#    book.append(albums)

In [52]:
#with open('picture_book.pkl', 'wb') as f:  
#    pickle.dump(book, f)

In [53]:
with open('picture_book.pkl', 'rb') as f:
    picture_book = pickle.load(f)

In [54]:
groups = response_df['patientID']
groups = np.tile(groups, len(picture_book))
#Make sure the indexes match - they should be 68
#np.sum(groups == patient_IDs[6])

In [55]:
y = response_df.values
y = response_df.drop(['patientID'], axis = 1).values
y = StandardScaler().fit_transform(y)
y = np.tile(y, (len(picture_book),1))

In [ ]:
#Select number of runs
nruns = 50
folds = 10
nomics = 8

#Using one omics at a time
prediction_train_f = []
observed_train_f = []
prediction_test_f = []
observed_test_f = []

for pred in tqdm(range(nomics)):
    
    X = []
    for chapter in range(len(picture_book)):
        X.append(np.asarray(picture_book[chapter][pred])) 

    X = np.vstack(X)

    #prediction_train, observed_train, 
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=nruns)
                           (delayed(butterfly.Models.CNN)
                            (X, y, groups, pixels, features, folds) 
                            for cv in range(nruns)))                

    prediction_train_f.append(prediction_train)
    observed_train_f.append(observed_train)
    prediction_test_f.append(prediction_test)
    observed_test_f.append(observed_test)




  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
statistic = 'spearman'

In [ ]:
each_omic_train = []
each_omic_test = []

for omic in range(nomics):    
    each_feature_train = []
    each_run_train = []
    
    each_feature_test = []
    each_run_test = []

    for runs in range(nruns):
        correl_train = prediction_train_f[omic][runs].corrwith(observed_train_f[omic][runs], axis = 0, method = statistic) 
        each_feature_train.append(correl_train)
        each_run_train.append(np.mean(each_feature_train))
        
        correl_test = prediction_test_f[omic][runs].corrwith(observed_test_f[omic][runs], axis = 0, method = statistic) 
        each_feature_test.append(correl_test)
        each_run_test.append(np.mean(each_feature_test))

    each_omic_train.append(np.mean(each_run_train))
    each_omic_test.append(np.mean(each_run_test))

In [ ]:
#Results - one omics at a time
#results = []
#for k in range(6):
#    correlation = []
#    for l in range(len(bars_p[k].columns)):
#        correlation.append(r2_score(bars_p[k][l], bars_o[k][l]))        
#    results.append(np.mean(correlation))

In [ ]:
y_pos = np.arange(nomics)

In [ ]:
#Plot the results
plt.bar(y_pos, each_omic_test, color=['firebrick', 'gold', 'olivedrab', 'royalblue', 
                                       'cyan', 'salmon', 'slategray', 'magenta'])
plt.xticks(y_pos, predictors)
plt.xlabel('Predictor')
plt.ylabel('Spearman r')
plt.title('Prediction accuracy for plasma l')
plt.show()

In [ ]:
X_m = [albums[0], albums[1], albums[2], albums[3], albums[4], albums[5], albums[6]]
del X_m[response_number]
X_m = np.array(X_m, dtype = float)

In [ ]:
#Using all omics at once
prediction_train_m, observed_train_m, prediction_test_m, observed_test_m = zip(*Parallel(n_jobs=3)(delayed(butterfly.Models.multi_CNN)
                                               (X_m, y, groups, pixels, features, folds) 
                                               for cv in tqdm(range(nruns))))

In [ ]:
each_feature_train_m = []
each_run_train_m = []

each_feature_test_m = []
each_run_test_m = []

for runs in range(nruns):
    correl_train_m = prediction_train_m[runs].corrwith(observed_train_m[runs], axis = 0, method = statistic) 
    each_feature_train_m.append(correl_train_m)
    each_run_train_m.append(np.mean(each_feature_train_m))

    correl_test_m = prediction_test_m[runs].corrwith(observed_test_m[runs], axis = 0, method = statistic) 
    each_feature_test_m.append(correl_test_m)
    each_run_test_m.append(np.mean(each_feature_test_m))
    
all_omics_train = np.mean(each_run_train_m)
all_omics_test = np.mean(each_run_test_m)

In [ ]:
#Results - all omics at once
each_omic_train.append(all_omics_train)
each_omic_test.append(all_omics_test)

In [ ]:
#Define the predictor datasets
predictors = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s', 'all']
y_pos = np.arange(len(predictors))

In [ ]:
#Plot the results
plt.bar(y_pos, each_omic_test, color=['firebrick', 'gold', 'olivedrab', 'royalblue', 
                                      'cyan', 'salmon', 'slategray', 'magenta'])
plt.xticks(y_pos, predictors)
plt.xlabel('Predictor')
plt.ylabel('Spearman r')
plt.title('Prediction accuracy for plasma l')
plt.show()

In [ ]:
print('test')